In [1]:
import os
import numpy as np
import pandas as pd
import nltk
import keras
import six
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,AveragePooling1D,Flatten,concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import *

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to /home/roberto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roberto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/roberto/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#Abro el archivo en el que se encuentra el dataset de los problemas
with open('singleop.json', 'r') as f:
    datastore = json.load(f)

In [4]:
# Archivo que contiene un listado de nombres
nombres = pd.read_csv('nombres-2015.csv')
names = pd.read_csv('yob2019.txt', header=None)    

In [5]:
dataset2 = pd.read_csv('problemas_adicionales2.csv')

In [6]:
preguntas = []
respuestas = []
ecuaciones = []
alineacion = []

for item in datastore:
    preguntas.append(item['sQuestion'])
    respuestas.append(item['lSolutions'])
    ecuaciones.append(item['lEquations'])
    alineacion.append(item['lAlignments'])

In [7]:
# Necesito convertir el dataset en un problema de clasificacion para que la red neuronal pueda identificar
# si estoy tratando de resolver un problema de sumas, restas, multiplicaciones o divisiones.
# Esto va a crear una lista con el tipo de operacion y que va a ser el resultado a inferir.
operaciones = []
sumas =0
restas =0
multiplicaciones =0
divisiones = 0
otras = 0
#Clasifico las operaciones en 0 para sumas, 1 para restas, 2 para multiplicaciones, 3 para divisiones y 4 sino lo encuentro.
for operacion in ecuaciones:
    if (operacion[0].find('+')>=0):
        operaciones.append(0)
        sumas = sumas + 1
    elif (operacion[0].find('-') >= 0 ):
        operaciones.append(1)
        restas = restas + 1
    elif(operacion[0].find('*') >=0):
        operaciones.append(2)
        multiplicaciones = multiplicaciones + 1
    elif(operacion[0].find('/')):
        operaciones.append(3)
        divisiones = divisiones + 1
    else:
        operaciones.append(4)
        otras = otras + 1

print('Tengo ', sumas, ' sumas ', restas, ' restas, ', multiplicaciones, ' multiplicaciones, ', divisiones, ' divisiones y otras operaciones ', otras)

Tengo  159  sumas  162  restas,  117  multiplicaciones,  124  divisiones y otras operaciones  0


In [8]:
preguntas2 = dataset2['Preguntas'].tolist()
respuestas2 = dataset2['respuestas'].tolist()

In [9]:
preguntas3 = preguntas + preguntas2
respuestas3 = operaciones + respuestas2

In [10]:
#El listado de nombres lo voy a truncar a los 15K primeros, dado que el resto son nombres muy residuales.
nombres_ = nombres['nombre'][:15000]

In [11]:
names_= names[0]

In [12]:
nombres_ = nombres_.append(names_)

In [13]:
st_words = list(stop_words)

In [20]:
nom = nombres['nombre'].values.tolist()

In [21]:
nomb =  nom

In [22]:
nomb[-1]

'Fiorella Luz Mailén'

In [23]:
# El vector preguntas_sin, consiste en las preguntas a las que voy a eliminar todos los nombres propios que no
# anaden ningun valor al conjunto de preguntas. No quiero que esos nombres se procesen y por tanto los elimino.
def eliminar_palabras(dataset, stopw):
    preguntas_sin = []
    for palabras in dataset:
        frases = [word for word in palabras.split(' ') if word not in stopw]
        frases = " ".join(frases)
        preguntas_sin.append(frases)
    return preguntas_sin

In [24]:
preguntas_sin = eliminar_palabras(preguntas3, nomb)

In [26]:
preguntas_sin = preguntas3 + eliminar_palabras(preguntas3, nomb) + eliminar_palabras(preguntas3, stop_words)

In [27]:
respuestas3 = respuestas3 *3

In [28]:
print(preguntas_sin[130], respuestas3[130])

Cada ticket cuesta $9.00. ¿Cuanto cuestan 4 tickets? 2


In [30]:
from sklearn.utils import shuffle

preguntas3, respuestas3 = shuffle(preguntas_sin,respuestas3)

In [31]:
preguntas_w = []
tokenizer = RegexpTokenizer(r'\w+')

for i,pregunta in enumerate(preguntas_sin):
    palabras = tokenizer.tokenize(pregunta)
    for palabra in palabras:
        preguntas_w.append(palabra)

In [32]:
palabras_unicas = set(preguntas_w)

In [33]:
p_unicas = list(palabras_unicas)

vocabulario = {p:i for i, p in enumerate(p_unicas)}

In [34]:
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

In [35]:
glove = gensim.models.KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.txt')

In [36]:
embeddings = []
for key in vocabulario.keys():
    if key not in glove:
        zeros = np.zeros(300)
        if key.isdigit():
            for i in range(0,300):
                zeros[i] = int(key)*0.000001
        embeddings.append({key: zeros})
    else:
        embeddings.append({key : glove[key][:300]})

In [37]:
p = np.zeros(300)

In [38]:
preguntas_embeddings = []

for i, preguntas in enumerate(preguntas_sin):
    palabras = tokenizer.tokenize(preguntas)
    words = []
    for j,palabra in enumerate(palabras):
        for h, word in enumerate(embeddings):
            for key in embeddings[h]:
                if(key == palabra):
                    words.append(embeddings[h][palabra][:300])
    if (j < 48):
        for j in range (j, 48):
            words.append(p)
    preguntas_embeddings.append(words)

In [39]:
# Calcular el tamano del vocabulario
vocabulario = []

for pregunta in preguntas_w:
    for palabra in pregunta:
        vocabulario.append(palabra)

In [40]:
vocabulario = set(vocabulario)

In [41]:
len(vocabulario)

69

In [42]:
# El tamano del vocabulario es necesario para poder crear la matriz de embeddings, con el numero de palabras
# totales que tengo
embedding_dim = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [43]:
len(preguntas_embeddings)

2922

In [44]:
training_X = np.asarray(preguntas_embeddings[:2100])
test_X = np.asarray(preguntas_embeddings[2100:])
training_y = np.asarray(respuestas3[:2100])
test_y = np.asarray(respuestas3[2100:])


In [45]:
for i in training_X:
    if(len(i) != 49):
        print(len(i))

In [46]:
# Creamos un modelo donde el primer argumento de la capa embedding son las palabras totales que voy a procesar
# vectorizadas en un indice.
# El segundo argumento, es el tamano del vector embedding, que he fijado en 16.
# El tercer argumento, es el tamano o longitud maxima, que he definido para las preguntas. Numero total de palabras
# por pregunta.
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [48]:
history = model.fit(training_X, training_y, epochs = 300, validation_data=(test_X, test_y), batch_size = 128)

Epoch 1/300
17/17 [==============================] - 1s 43ms/step - loss: 1.3622 - accuracy: 0.3329 - val_loss: 1.3507 - val_accuracy: 0.3783
Epoch 2/300
17/17 [==============================] - 0s 22ms/step - loss: 1.3332 - accuracy: 0.3629 - val_loss: 1.3512 - val_accuracy: 0.3783
Epoch 3/300
17/17 [==============================] - 0s 23ms/step - loss: 1.3053 - accuracy: 0.3762 - val_loss: 1.3471 - val_accuracy: 0.3820
Epoch 4/300
17/17 [==============================] - 0s 25ms/step - loss: 1.2666 - accuracy: 0.4033 - val_loss: 1.3468 - val_accuracy: 0.3929
Epoch 5/300
17/17 [==============================] - 0s 26ms/step - loss: 1.2171 - accuracy: 0.4376 - val_loss: 1.3522 - val_accuracy: 0.3771
Epoch 6/300
17/17 [==============================] - 0s 25ms/step - loss: 1.1427 - accuracy: 0.4876 - val_loss: 1.3511 - val_accuracy: 0.3431
Epoch 7/300
17/17 [==============================] - 0s 25ms/step - loss: 1.0791 - accuracy: 0.5390 - val_loss: 1.3866 - val_accuracy: 0.3175
Epoch 

17/17 [==============================] - 0s 16ms/step - loss: 0.3758 - accuracy: 0.7705 - val_loss: 2.6710 - val_accuracy: 0.3236
Epoch 59/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3719 - accuracy: 0.7748 - val_loss: 2.7841 - val_accuracy: 0.3151
Epoch 60/300
17/17 [==============================] - 0s 22ms/step - loss: 0.3784 - accuracy: 0.7848 - val_loss: 2.7361 - val_accuracy: 0.3333
Epoch 61/300
17/17 [==============================] - 0s 18ms/step - loss: 0.3737 - accuracy: 0.7719 - val_loss: 2.8491 - val_accuracy: 0.3187
Epoch 62/300
17/17 [==============================] - 0s 20ms/step - loss: 0.3772 - accuracy: 0.7767 - val_loss: 2.7589 - val_accuracy: 0.3090
Epoch 63/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3669 - accuracy: 0.7852 - val_loss: 2.8715 - val_accuracy: 0.3260
Epoch 64/300
17/17 [==============================] - 0s 20ms/step - loss: 0.3607 - accuracy: 0.7724 - val_loss: 3.0034 - val_accuracy: 0.3163
Epoch 65/300

17/17 [==============================] - 0s 25ms/step - loss: 0.3294 - accuracy: 0.7895 - val_loss: 3.4969 - val_accuracy: 0.3260
Epoch 116/300
17/17 [==============================] - 0s 18ms/step - loss: 0.3319 - accuracy: 0.7924 - val_loss: 3.6063 - val_accuracy: 0.3297
Epoch 117/300
17/17 [==============================] - 0s 20ms/step - loss: 0.3292 - accuracy: 0.7862 - val_loss: 3.6680 - val_accuracy: 0.3333
Epoch 118/300
17/17 [==============================] - 0s 22ms/step - loss: 0.3224 - accuracy: 0.7914 - val_loss: 3.7386 - val_accuracy: 0.3127
Epoch 119/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3259 - accuracy: 0.7752 - val_loss: 3.7195 - val_accuracy: 0.3151
Epoch 120/300
17/17 [==============================] - 0s 19ms/step - loss: 0.3306 - accuracy: 0.7933 - val_loss: 3.6092 - val_accuracy: 0.3163
Epoch 121/300
17/17 [==============================] - 0s 23ms/step - loss: 0.3268 - accuracy: 0.7814 - val_loss: 3.7989 - val_accuracy: 0.3260
Epoch 

Epoch 172/300
17/17 [==============================] - 0s 23ms/step - loss: 0.3208 - accuracy: 0.7838 - val_loss: 4.4105 - val_accuracy: 0.3260
Epoch 173/300
17/17 [==============================] - 0s 22ms/step - loss: 0.3173 - accuracy: 0.7848 - val_loss: 4.3692 - val_accuracy: 0.3187
Epoch 174/300
17/17 [==============================] - 0s 19ms/step - loss: 0.3202 - accuracy: 0.7886 - val_loss: 4.5421 - val_accuracy: 0.3236
Epoch 175/300
17/17 [==============================] - 0s 22ms/step - loss: 0.3257 - accuracy: 0.7852 - val_loss: 4.1245 - val_accuracy: 0.3248
Epoch 176/300
17/17 [==============================] - 0s 23ms/step - loss: 0.3158 - accuracy: 0.7910 - val_loss: 4.5214 - val_accuracy: 0.3151
Epoch 177/300
17/17 [==============================] - 0s 23ms/step - loss: 0.3184 - accuracy: 0.7795 - val_loss: 4.6024 - val_accuracy: 0.3114
Epoch 178/300
17/17 [==============================] - 0s 25ms/step - loss: 0.3120 - accuracy: 0.7981 - val_loss: 4.4530 - val_accuracy:

Epoch 229/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3105 - accuracy: 0.7824 - val_loss: 5.2668 - val_accuracy: 0.3285
Epoch 230/300
17/17 [==============================] - 0s 24ms/step - loss: 0.3161 - accuracy: 0.7862 - val_loss: 5.2176 - val_accuracy: 0.3090
Epoch 231/300
17/17 [==============================] - 0s 24ms/step - loss: 0.3136 - accuracy: 0.8048 - val_loss: 5.1438 - val_accuracy: 0.3041
Epoch 232/300
17/17 [==============================] - 0s 26ms/step - loss: 0.3130 - accuracy: 0.7862 - val_loss: 5.2152 - val_accuracy: 0.3175
Epoch 233/300
17/17 [==============================] - 0s 27ms/step - loss: 0.3185 - accuracy: 0.7810 - val_loss: 5.3856 - val_accuracy: 0.3248
Epoch 234/300
17/17 [==============================] - 0s 19ms/step - loss: 0.3112 - accuracy: 0.7767 - val_loss: 5.2900 - val_accuracy: 0.3102
Epoch 235/300
17/17 [==============================] - 0s 16ms/step - loss: 0.3123 - accuracy: 0.7929 - val_loss: 5.3459 - val_accuracy:

Epoch 286/300
17/17 [==============================] - 0s 25ms/step - loss: 0.3103 - accuracy: 0.7976 - val_loss: 5.5843 - val_accuracy: 0.3200
Epoch 287/300
17/17 [==============================] - 0s 19ms/step - loss: 0.3090 - accuracy: 0.7962 - val_loss: 5.6604 - val_accuracy: 0.3163
Epoch 288/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3200 - accuracy: 0.7790 - val_loss: 5.6596 - val_accuracy: 0.3309
Epoch 289/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3187 - accuracy: 0.7795 - val_loss: 5.5346 - val_accuracy: 0.3041
Epoch 290/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3176 - accuracy: 0.7886 - val_loss: 5.8443 - val_accuracy: 0.3309
Epoch 291/300
17/17 [==============================] - 0s 18ms/step - loss: 0.3077 - accuracy: 0.7900 - val_loss: 5.7608 - val_accuracy: 0.3163
Epoch 292/300
17/17 [==============================] - 0s 21ms/step - loss: 0.3086 - accuracy: 0.7929 - val_loss: 5.9364 - val_accuracy:

In [49]:
model2 = keras.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(128, dropout = 0.2, return_sequences= True, input_shape=training_X.shape)),
    keras.layers.LSTM (64, dropout = 0.2, return_sequences = True),
    keras.layers.LSTM (128, dropout = 0.2, return_sequences = True),
    keras.layers.LSTM (256, dropout = 0.2, return_sequences = True),
    keras.layers.LSTM (64, dropout = 0.2),
    keras.layers.Dense(5, activation='softmax')
])
opt = keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
)

#opt = keras.optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
#opt = keras.optimizers.Adam(learning_rate = 0.0001)
model2.compile(loss='sparse_categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
history2 = model2.fit(training_X, training_y, epochs = 240, batch_size= 128, validation_data=(test_X, test_y))

Epoch 1/240
17/17 [==============================] - 24s 1s/step - loss: 0.7213 - accuracy: 0.6614 - val_loss: 2.4714 - val_accuracy: 0.2628
Epoch 2/240
17/17 [==============================] - 27s 2s/step - loss: 0.7167 - accuracy: 0.6600 - val_loss: 2.5147 - val_accuracy: 0.2835
Epoch 3/240
17/17 [==============================] - 22s 1s/step - loss: 0.6866 - accuracy: 0.6614 - val_loss: 2.5142 - val_accuracy: 0.2883
Epoch 4/240
17/17 [==============================] - 22s 1s/step - loss: 0.7306 - accuracy: 0.6462 - val_loss: 2.2663 - val_accuracy: 0.3260
Epoch 5/240
17/17 [==============================] - 30s 2s/step - loss: 0.6956 - accuracy: 0.6757 - val_loss: 2.2925 - val_accuracy: 0.3163
Epoch 6/240
17/17 [==============================] - 27s 2s/step - loss: 0.7254 - accuracy: 0.6305 - val_loss: 2.5769 - val_accuracy: 0.2701
Epoch 7/240
17/17 [==============================] - 24s 1s/step - loss: 0.6891 - accuracy: 0.6638 - val_loss: 2.3437 - val_accuracy: 0.3127
Epoch 8/240
1

Epoch 59/240
17/17 [==============================] - 29s 2s/step - loss: 0.5862 - accuracy: 0.6976 - val_loss: 2.9251 - val_accuracy: 0.2956
Epoch 60/240
17/17 [==============================] - 27s 2s/step - loss: 0.5711 - accuracy: 0.7095 - val_loss: 2.6352 - val_accuracy: 0.2908
Epoch 61/240
17/17 [==============================] - 24s 1s/step - loss: 0.6085 - accuracy: 0.6829 - val_loss: 2.8145 - val_accuracy: 0.3029
Epoch 62/240
17/17 [==============================] - 25s 1s/step - loss: 0.5752 - accuracy: 0.7181 - val_loss: 2.8610 - val_accuracy: 0.2713
Epoch 63/240
17/17 [==============================] - 26s 2s/step - loss: 0.5771 - accuracy: 0.7019 - val_loss: 2.9314 - val_accuracy: 0.2859
Epoch 64/240
17/17 [==============================] - 26s 2s/step - loss: 0.5703 - accuracy: 0.7014 - val_loss: 2.7369 - val_accuracy: 0.3163
Epoch 65/240
17/17 [==============================] - 29s 2s/step - loss: 0.5491 - accuracy: 0.7162 - val_loss: 3.1270 - val_accuracy: 0.3041
Epoch 